# Загрузка Pandas и очистка данных

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('main_task.csv')

In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [ ]:
import numpy as np
import re
from datetime import datetime, timedelta

In [ ]:
df['Cuisine Style'].fillna("['Not data']", axis=0, inplace=True) #замена пропусков кол-во кухонь, нужно изменить

In [ ]:
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x[2:-2])
df['Cuisine Style'] = df['Cuisine Style'].str.split("', '")

In [ ]:
cuisins = set()
for cuisins_style in df['Cuisine Style']:
    for cuisin in cuisins_style:
        cuisins.add(cuisin)

len(cuisins) #кол-во стилей кухонь, +1 - нет данных

In [ ]:
def find_item(cell):
    if item in cell:
        return 1
    return 0

In [ ]:
for item in cuisins:
    df[item] = df['Cuisine Style'].apply(find_item)

In [ ]:
df['Many cuisins'] = df['Cuisine Style'].apply(lambda x: len(x)) #добавляем показатель с кол-вом стилей кухонь в каждом ресторане

In [ ]:
df['Date reviews'] = df['Reviews'].apply(lambda x: re.findall(r'\d{2}/\d{2}/\d{4}', x)) #выделяем даты отзывов

In [ ]:
for i in range(len(df['Date reviews'])):
    df['Date reviews'][i] = [datetime.strptime(x, '%m/%d/%Y') for x in df['Date reviews'][i]] #преобразовываем в тип datetime

In [ ]:
import time

for i in range(len(df['Date reviews'])):
    df['Date reviews'][i] = list(map(datetime.timestamp, df['Date reviews'][i]))

In [ ]:
df['Date reviews']

In [ ]:
#добавляем признак даты последнего отзыва, в днях с 1970г
df['Date last review'] = df.apply(lambda x: max(x['Date reviews']) if len(x['Date reviews']) > 0 else 0, axis=1)

In [ ]:
df['Date last review'].head(5)

In [ ]:
#добавить признак с id города
df['City ID'] = df['City'].map({'London': 4420, 'Paris': 331, 'Madrid':341, 'Barcelona':343,
                               'Berlin':4930, 'Milan':392, 'Rome':396, 'Prague':4202,
                               'Lisbon':35121, 'Vienna':431, 'Amsterdam':3120, 'Brussels':322,
                               'Hamburg':4940, 'Munich':4989, 'Lyon':33437, 'Stockholm':468,
                               'Budapest':361, 'Warsaw':4822, 'Dublin':3531, 'Copenhagen':451,
                               'Athens':30210, 'Edinburgh':44131, 'Zurich':411, 'Oporto':3512,
                               'Geneva':4122, 'Krakow':4812, 'Oslo':4722, 'Helsinki':3589,
                               'Bratislava':4212, 'Luxembourg':352, 'Ljubljana':3861}) 

In [ ]:
#добавить признак с населением города
df['City population'] = df['City'].map({'London': 8.982, 'Paris': 2.140, 'Madrid':6.642, 'Barcelona':5.575,
                                       'Berlin':3.769, 'Milan':1.332, 'Rome':2.870, 'Prague':1.309,
                                       'Lisbon':0.507, 'Vienna':1.897, 'Amsterdam':0.825, 'Brussels':1.209,
                                       'Hamburg':1.841, 'Munich':1.472, 'Lyon':0.496, 'Stockholm':0.976,
                                       'Budapest':1.752, 'Warsaw':1.791, 'Dublin':1.388, 'Copenhagen':1.247,
                                       'Athens':3.169, 'Edinburgh':0.525, 'Zurich':0.402, 'Oporto':0.222,
                                       'Geneva':0.499, 'Krakow':0.779, 'Oslo':0.681, 'Helsinki':0.655,
                                       'Bratislava':0.438, 'Luxembourg':0.122, 'Ljubljana':0.284}) 

In [ ]:
#добавить признак с площадью города
df['City square'] = df['City'].map({'London': 1572, 'Paris': 105.4, 'Madrid':604.3, 'Barcelona':101.9,
                                    'Berlin':891.8, 'Milan':181.8, 'Rome':1285, 'Prague':496,
                                    'Lisbon':100, 'Vienna':414.6, 'Amsterdam':219.3, 'Brussels':32.61,
                                    'Hamburg':755.2, 'Munich':310.4, 'Lyon':47.87, 'Stockholm':188,
                                    'Budapest':525.2, 'Warsaw':517.2, 'Dublin':115, 'Copenhagen':88.25,
                                    'Athens':2929, 'Edinburgh':264, 'Zurich':87.88, 'Oporto':41.42,
                                    'Geneva':15.93, 'Krakow':327, 'Oslo':454, 'Helsinki':213.8,
                                    'Bratislava':367.6, 'Luxembourg':51.46, 'Ljubljana':163.8}) 

In [ ]:
def fillna_number_reviews(row):
    if np.isnan(row['Number of Reviews']):
        row['Number of Reviews'] = df[df['City'] == row['City']]['Number of Reviews'].mean()
        return row['Number of Reviews']
    return row['Number of Reviews']


df['Number of Reviews'] = df.apply(lambda row: fillna_number_reviews(row), axis=1)

In [ ]:
df.head()

In [ ]:
df = df.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 'Date reviews'], axis=1) #удаление не числовых признаков

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis=1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = np.round(regr.predict(X_test), 1)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))